<a href="https://colab.research.google.com/github/torontodeveloper/berkeley-ai-llm-foundations/blob/main/KevinKakollaRecurrentNeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Name: Kevin Kakolla
# Student Number: X257515

In [ ]:
#import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf

In [ ]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.debugging.set_log_device_placement(True)


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nageswarraokakolla","key":"548ab64ccef1add72a9d2663b0b48f98"}'}

In [ ]:
# Make a directory named .kaggle
!mkdir -p ~/.kaggle

# Copy the kaggle.json to this new directory
!cp kaggle.json ~/.kaggle/

# Change the permissions of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Install Kaggle API
!pip install kaggle

# Download the dataset
!kaggle datasets download -d shivamb/real-or-fake-fake-jobposting-prediction

Dataset URL: https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction
License(s): CC0-1.0
 31% 5.00M/16.1M [00:00<00:00, 51.6MB/s]
100% 16.1M/16.1M [00:00<00:00, 118MB/s] 


In [ ]:
!kaggle datasets download -d shivamb/real-or-fake-fake-jobposting-prediction

Dataset URL: https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction
License(s): CC0-1.0
real-or-fake-fake-jobposting-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/real-or-fake-fake-jobposting-prediction.zip

Archive:  /content/real-or-fake-fake-jobposting-prediction.zip
  inflating: fake_job_postings.csv   


In [ ]:
fake_postings_df = pd.read_csv('/content/fake_job_postings.csv')

In [ ]:
if tf.test.gpu_device_name():
  print('GPU device name',tf.test.gpu_device_name())
else:
  print('Install tf gpu version',tf.__version__)

GPU device name /device:GPU:0


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
print('Tensorflow version',tf.__version__)

Tensorflow version 2.17.1


In [ ]:
fake_postings_df=fake_postings_df[:10000]

In [ ]:
fake_postings_df.shape

(10000, 18)

In [ ]:
fake_postings_df.fillna('',inplace=True)

<ipython-input-16-70f95e89b1de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_postings_df.fillna('',inplace=True)


In [ ]:
fake_postings_df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [ ]:
text_columns=['title','location', 'department','benefits','employment_type','required_experience','industry','function','company_profile','description','requirements']
fake_postings_df['combined_text'] = fake_postings_df[text_columns].fillna('').apply(lambda x:' '.join(x),axis=1)

<ipython-input-18-0275e8dda8be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_postings_df['combined_text'] = fake_postings_df[text_columns].fillna('').apply(lambda x:' '.join(x),axis=1)


In [ ]:
fake_postings_df = fake_postings_df.drop(['title','location','department','benefits','employment_type','required_experience','industry','function','company_profile','description','requirements'],axis=1)

In [ ]:
fake_postings_df.head(2)

,job_id,salary_range,telecommuting,has_company_logo,has_questions,required_education,fraudulent,combined_text
0,1,,0,1,0,,0,"Marketing Intern US, NY, New York Marketing O..."
1,2,,0,1,0,,0,"Customer Service - Cloud Video Production NZ, ..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

numeric_columns = fake_postings_df.columns.difference(['combined_text','fraudulent'])
X_numeric = fake_postings_df[numeric_columns]

In [ ]:
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(fake_postings_df['combined_text'].fillna(''))

In [ ]:
X_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   has_company_logo    10000 non-null  int64 
 1   has_questions       10000 non-null  int64 
 2   job_id              10000 non-null  int64 
 3   required_education  10000 non-null  object
 4   salary_range        10000 non-null  object
 5   telecommuting       10000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 468.9+ KB


In [ ]:
X_numeric = X_numeric.apply(pd.to_numeric,errors='coerce')

In [ ]:
X_numeric=X_numeric.fillna(0)

In [ ]:
import scipy.sparse as sp

X_combined = sp.hstack([X_text, sp.csr_matrix(X_numeric.values)], format='csr')

In [ ]:
y=fake_postings_df['fraudulent']
X = fake_postings_df.drop(['fraudulent'],axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_combined,y,test_size=0.2,random_state=42,shuffle=True)

In [ ]:
y_train.shape,y_test.shape

((8000,), (2000,))

In [ ]:
X_train.shape,X_test.shape

((8000, 77068), (2000, 77068))

In [ ]:
vocab_size=20000
embedding_dim = 32
sequence_length=20

In [ ]:
model = keras.models.Sequential([
                                keras.layers.LSTM(
                                    1,
                                    input_shape=[None,1],
                                    activation='relu',
                                    return_sequences= True
                                  ),
                                 keras.layers.TimeDistributed(
                           keras.layers.Dense(1, activation='sigmoid',use_bias=True))
                                 ])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op FloorMod in device /job:localhost/replica:0/task:0/device:GPU:0


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device

In [ ]:
def last_time_step_mse(Y_true,Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:,-1],Y_pred[:,-1])

In [ ]:
model.compile(loss='mse',optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics=[last_time_step_mse])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, None, 1)             │              12 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, None, 1)             │               2 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14 (56.00 B)

 Trainable params: 14 (56.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
!nvidia-smi

Mon Dec 23 03:39:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              54W / 400W |    427MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
with tf.device('/GPU:0'):
  history = model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in devi

In [ ]:
# let's examine the learning curves
def plot_learning_curves(history):

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Mean Squared Error')
  plt.ylabel('MSE')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

plot_learning_curves(history)

In [ ]:
model.summary()

In [ ]:
model.evaluate(X_test,y_test,batch_size=32)

In [ ]:
model.predict(X_test[0])[-1].shape

In [ ]:
# a plot of our predictions will give us a better impression
# of how we did
def plot_forecast(model, X_sample, y_sample):

    plt.plot([x for x in range(50)], X_sample, marker=".")
    plt.plot([x for x in range(50,60)], y_sample, color="red", marker="o")
    plt.plot([x for x in range(50,60)], model.predict(X_sample)[-1], color="blue", marker="x")
    plt.xlabel("t")
    plt.ylabel("x(t)")
    plt.legend(["Input", "Actual", "Forecast"])
    plt.show()


plot_forecast(model, X_test[0], y_test[0])